# MPI tutorial

This notebook comes in two pieces: server-side (this one) and client-side ([Client.ipynb](Notebook-02-Client.ipynb)). The two notebooks are meant to be executed side-by-side.

## Disclaimer

The approach used in this tutorial, i.e. using two notebooks connected to each other, it is not the standard nor the suggested approach for using MPI. Usually one would have a python script `script.py` that is run on `N` processes via

```
mpirun -n N python script.py
```

as in the following:

In [ ]:
%%writefile mpi_example.py

from mpi4py import MPI

comm = MPI.COMM_WORLD

print(f"Hello from rank {comm.rank}/{comm.size}")

In [ ]:
! mpirun -n 8 python mpi_example.py

## Server-side

### Step-1

The notebooks need to be executed with `mpirun`. This can be enabled with a custom Jupyter kernel. First one needs to unpack the content of [MPIpython3.zip](MPIpython3.zip) into appropriate directory:

- Linux: `~/.local/share/jupyter/kernels`
- Mac: `~/Library/Jupyter/kernels`
- Windows: `%APPDATA%\jupyter\kernels`

And then restart the Jupyter notebook.

### Step-2.1

Select the kernel `MPI Python 3` for the notebook: kernel -> Change kernel...

Same for [Client.ipynb](Notebook-02-Client.ipynb).

### Step-3.1

Execute the following cell and after in [Client.ipynb](Notebook-02-Client.ipynb).

In [ ]:
from mpi4py import MPI

port = MPI.Open_port()
open(".port", "w").write(port)

comm = MPI.COMM_WORLD.Accept(port)
MPI.Close_port(port)

comm

### Step-4

Now the notebooks have been connected and you can proceed with the examples.


## Example 1: Hello

Since we have built an `intercomm`, both processes have rank 0.

In [ ]:
comm.rank

In a normal situation, each process would have different rank. 

In [ ]:
dest = 0
comm.send("Hello from Server", dest)
comm.recv()

## Example 2: Tag

Tags can be used for identifying the messages. They must be `int`

In [ ]:
comm.send("message 1", dest, tag=1)
comm.send("message 2", dest, tag=2)
comm.send("message 3", dest, tag=3)

## Example 3: Status

The status collects information about the message

In [ ]:
status = MPI.Status()


def print_status(s):
    print(f"Received message from {s.source} with tag {s.tag} and size {s.count}B")


print(comm.recv(status=status))
print_status(status)

## Example 4: Array

Arrays, like many other Python objects, can be easily sent via MPI

In [ ]:
import numpy as np

arr = np.random.rand(10)
comm.send(arr, 0)
arr

But this is the "slow" way because the objects are pickled and much more data than needed is sent.

In [ ]:
arr = comm.recv(status=status)
print_status(status)
arr

In [ ]:
arr.nbytes

Using `Send` and `Recv`, instead only the array's content is sent. Here it is important though to previously allocate an array with the correct size and data type.

In [ ]:
arr = np.zeros(1)
comm.Recv(arr, status=status)
print_status(status)
arr

## Example 5: Non-blocking

In [ ]:
req = comm.irecv()
req

In [ ]:
arr = req.wait()
arr

## Example 6: Matrix Vector product

Consider,
$$A x = y,$$
which can be split in domains as
$$ \left({\begin{matrix}A_1&A_2\\A_3&A_4\end{matrix}}\right) \left({\begin{matrix}x_1\\x_2\end{matrix}}\right) = \left({\begin{matrix}A_1 x_1 + A_2 x_2\\A_3 x_1 + A_4 x_2\end{matrix}}\right) = \left({\begin{matrix}y_1\\y_2\end{matrix}}\right). $$

In this example we keep the top part of matrix and vector on this notebook and the bottom part on the client.

In [ ]:
n = 20
m1 = n // 2
m2 = n - n // 2

A = np.random.rand(m1, n)
x1 = np.random.rand(m1)
x2 = np.zeros(m2)

comm.Send(x1, dest)
comm.Recv(x2, dest)

y = A[:, :m1].dot(x1) + A[:, m1:].dot(x2)
y

In [ ]:
comm.Isend(x1, dest)
req = comm.Irecv(x2, dest)

y = A[:, :m1].dot(x1)

req.wait()

y += A[:, m1:].dot(x2)
y